# `PointsDirectory` - A class used to encapsulate all calculations for many geometries (of a dataset)

The `ichor.core.files.PointsDirectory` class can be used to easily work with thousands of files which are generated when getting Gaussian, AIMAll, etc. calculations for many geometries.

The general structure of a `PointsDirectory`-like directory is like so:

```
.
|--- SYSTEM0001.pointdir
|   |--- SYSTEM0001_atomicfiles
|   |   |--- h2.int
|   |   |--- h3.int
|   |   |--- o1.int
|   |--- SYSTEM0001.gjf
|   |--- SYSTEM0001.wfn
|--- SYSTEM0002.pointdir
|   |--- SYSTEM0002_atomicfiles
|   |   |--- h2.int
|   |   |--- h3.int
|   |   |--- o1.int
|   |--- SYSTEM0002.gjf
|   |--- SYSTEM0002.wfn
...
...
...
```

Essentially, the `PointsDirectory` is a classed that is used to parse a directory contains many sub-directories (which are instances of `PointDirectory`). Each sub-directory (e.g. `SYSTEM0001.pointdir`, `SYSTEM0002.pointdir`) contains all relevant calculations for **one** molecular geometry. Each of the *sub-directories* can be individually read in as a `ichor.core.files.PointDirectory` instance (note that there is no *s* in this case.)

This class makes it easy to access calculations for many geometries very easily.

## `PointDirectory` strucutre

The `PointDirectory` class encapsulates a directory, containing all relevant calculations for **one** geometry. It subclasses from `ichor.core.files.directory.AnnotatedDirectory`. This gives us the ability to define *class* variables, which are of specific file types. Then the `AnnotatedDirectory._parse` method is what parses all files in the directory. The extensions of the files determine what the file type, and thus the class which is going to be used to parse the file.

The `PointDirectory.contents` class variable can be overwritten to quickly add support for new file and directory file types. This ensures that any new file or directory types in ichor are ready to be used with `PointDirectory`. The `contents` variable is a Python dictionary containing keys which are going to available as attributes after parsing, and values containing the Python class which is going to parse the relevant file or directory. For example, this is the current `contents` variable:

```python
contents = {
    "xyz": XYZ,
    "gjf": GJF,
    "gaussian_output": GaussianOutput,
    "orca_input": OrcaInput,
    "orca_output": OrcaOutput,
    "aim": Aim,
    "wfn": WFN,
    "ints": IntDirectory,
}
```

where `XYZ` is the class that is going to read a `.xyz` file in the directory.



## Obtaining results from a `PointsDirectory`

### Obtaining total system energy

The following code snippet can be used to quickly get the total system energy from a Gaussian calculation for example

In [5]:
from ichor.core.files import PointsDirectory

# PointsDirectory("path_to_directory_with_wfn_and_int_files")
points_dir = PointsDirectory("../../../example_files/example_points_directory/WATER_MONOMER.pointsdir")

for point_directory in points_dir:

    print(point_directory.name, point_directory.wfn.total_energy)

WATER_MONOMER0000.pointdir -76.421710687455
WATER_MONOMER0001.pointdir -76.429947804
WATER_MONOMER0002.pointdir -76.430599107417
WATER_MONOMER0003.pointdir -76.42948849797


### Accessing IQA energy for a specific atom

In [6]:
for point_directory in points_dir:

    print(point_directory.name, point_directory.ints["O1"].iqa)

# note that this is for A A'

WATER_MONOMER0000.pointdir -75.446714709
WATER_MONOMER0001.pointdir -75.453164031
WATER_MONOMER0002.pointdir -75.453749708
WATER_MONOMER0003.pointdir -75.453284702


### Accessing Mulipole Moments

In [7]:
for point_directory in points_dir:

    print(point_directory.name, point_directory.ints["O1"].global_spherical_multipoles)

# note these are not rotated

WATER_MONOMER0000.pointdir {'q00': -1.051921199, 'q10': -0.020042356378, 'q11c': 0.0018273275449, 'q11s': -0.20706556929, 'q20': -0.037266216811, 'q21c': -0.79780613831, 'q21s': 0.013146921148, 'q22c': -0.19595266005, 'q22s': 0.078488472227, 'q30': 0.043015515207, 'q31c': -0.053621704828, 'q31s': 0.21644282193, 'q32c': -0.029607236961, 'q32s': -0.89197505111, 'q33c': -0.053969314597, 'q33s': 0.16211677693, 'q40': -1.4545843935, 'q41c': 0.91783517331, 'q41s': 0.17650015949, 'q42c': -0.73112185714, 'q42s': -0.3293114897, 'q43c': 2.8344280941, 'q43s': -0.16267842746, 'q44c': -1.3853362266, 'q44s': 0.089771195512, 'q50': -0.24411738335, 'q51c': 0.48960856702, 'q51s': -1.5472642317, 'q52c': -0.040094542612, 'q52s': 0.98097072569, 'q53c': 0.72718022845, 'q53s': -1.1988409017, 'q54c': -0.47766441277, 'q54s': 2.0753064137, 'q55c': -0.29405113415, 'q55s': -1.6430303594}
WATER_MONOMER0001.pointdir {'q00': -1.1248310833, 'q10': -0.15773618224, 'q11c': 0.081543820356, 'q11s': 0.12130191092, 'q20':

### Accessing all data from all files

There is a very quick way to obtain all raw data from all calculations in a `PointsDirectory`. The `raw_data` property can be used to obtain the raw data. This returns a Python dictionary where the keys are the point names and the values are a nested Python dictionary containing the results from all the relevant calculations.

In [8]:
all_raw_data = points_dir.raw_data

all_raw_data

{'WATER_MONOMER0000': {'gaussian_output': {'global_forces': {'O1': array([ 0.02953315,  0.0827204 , -0.02495305]),
    'H2': array([ 0.00578961, -0.0242831 , -0.00842433]),
    'H3': array([-0.03532276, -0.05843731,  0.03337739])},
   'charge': 0,
   'multiplicity': 1,
   'molecular_dipole': MolecularDipole(x=0.1189, y=2.3866, z=0.0787),
   'molecular_quadrupole': MolecularQuadrupole(xx=-6.5273, yy=-7.7674, zz=-6.2577, xy=0.0665, xz=-1.6318, yz=-0.0495),
   'traceless_molecular_quadrupole': TracelessMolecularQuadrupole(xx=0.3235, yy=-0.9166, zz=0.5931, xy=0.0665, xz=-1.6318, yz=-0.0495),
   'molecular_octupole': MolecularOctupole(xxx=0.5348, yyy=8.5805, zzz=0.2229, xyy=0.1794, xxy=3.083, xxz=0.0727, xzz=0.1817, yzz=3.0764, yyz=0.0143, xyz=0.0059),
   'molecular_hexadecapole': MolecularHexadecapole(xxxx=-8.283, yyyy=-15.2596, zzzz=-8.1733, xxxy=-0.2961, xxxz=-0.1375, yyyx=-0.296, yyyz=0.056, zzzx=-0.1975, zzzy=0.0334, xxyy=-3.888, xxzz=-2.4818, yyzz=-3.8474, xxyz=-0.0404, yyxz=-0.2009, 

## Converting to SQLite3 database

Reading thousands of files every time is very time consuming (especially on hard drives), so it is much more efficient to read the data once and store it in a database. `ichor` has SQLite3 support implemented, meaning a `PointsDirecotry` can be readily converted to an SQLite3 database. **NOTE: ONLY RAW DATA FROM CALCULATIONS IS STORED IN THE DATABSE. NO POSTPROCESSING IS DONE. ANY POSTPROCESSING MUST BE DONE AT A LATER STEP (e.g. rotating multipole moments).**

Code snipped to produce database:

```python

from ichor.core.files import PointsDirectory

pd = PointsDirectory("points_directory_path")
pd.write_to_sqlite3_database()
```

**Note 1: It takes a while to read all files, so this should be submitted on compute.**

**Note 2: If the dataset is large and split into many `PointsDirectory`-like directories, then you can do**

```python
from ichor.core.files import PointsDirectory
from pathlib import Path

parent_dir = Path("parent_dir")

for d in parent_dir.iterdir():

    pd = PointsDirectory("points_directory_path")
    pd.write_to_sqlite3_database("large_database.db")
```

where all the information will be stored into one database.

## SQLite Database Schema Diagram

The following is that the schema diagram looks like for the table currently. The image was made with DBVisualizer. Note that these **all** fields might not be populated if the database. That depends on the raw data that is present in the `PointsDirectory`. For example, if only Gaussian are ran, then the AIMAll-related data will be missing from the database.

Below is a diagram of the SQLite3 Database, made with DbVisualizer
![alt text](../../../example_files/sql_database_schema.svg "SQLite3 Schema")

## Converting to JSON database

Very similarly, the `PointsDirectory` instance can be converted to a json database by

```python

from ichor.core.files import PointsDirectory

pd = PointsDirectory("points_directory_path")
pd.write_to_json_database()
```

## Generating CSV files with Features from SQLite3 Database

CSV files can be readily made from a `PointsDirectory` instance or a database. CSV files containing (ALF) features and relevant outputs can be generated from an SQLite3 database like so:

```python
from ichor.core.database.sql.query_database import (
    get_alf_from_first_db_geometry,
    write_processed_data_for_atoms_parallel,
    write_processed_data_for_atoms
)

db_path = "DATABASE_PATH"

# note that you can also define an ALF manually as well
# or get it from some other molecular geometry
# that contains the same atom sequencing as in the database
alf = get_alf_from_first_db_geometry(db_path)

# note that this will write files out in parallel
# use write_processed_data_for_atoms for serial

write_processed_data_for_atoms_parallel(
    db_path,
    alf,
    ncores=4,
    calc_multipoles=True, # rotates multipoles using C matrix
    calc_forces=False, # calculates ALF forces using Wilson B matrix
)
```